# ***Demo: Distributed training with Amazon SageMaker***

In [1]:
# !pip install sagemaker --upgrade -q
# !pip install ipywidgets -q
# !wget https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/tensorflow_bring_your_own/utils/generate_cifar10_tfrecords.py
# !python generate_cifar10_tfrecords.py --data-dir cifar10

**Step 1:** Import essentials packages, start a sagemaker session and specify the bucket name you created in the pre-requsites section of this workshop.

In [2]:
import os
import boto3
import time
import numpy as np
import sagemaker

sess = boto3.Session()
sm   = sess.client('sagemaker')
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket_name    = sagemaker_session.default_bucket()
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

#### Upload dataset if it doesn't exist

In [3]:
# datasets = sagemaker_session.upload_data(path='cifar10', key_prefix=f'{dataset_folder}/cifar10-dataset')
# datasets

#If dataset already exists
datasets = f's3://{bucket_name}/{dataset_folder}/cifar10-dataset'
datasets

's3://sagemaker-us-west-2-453691756499/datasets/cifar10-dataset'

**Step 3:** In this cell we create a SageMaker estimator, by providing it with all the information it needs to launch instances and execute training on those instances.

We specify `distributions` to SMDataParallel.

In the TensorFlow estimator call, we specify training script under `entry_point` and dependencies under `code`. SageMaker automatically copies these files into a TensorFlow container behind the scenes, and are executed on the training instances.

In [4]:
instance_type = 'ml.p3.16xlarge'
instance_count = 1

distribution={'smdistributed':{
                    'dataparallel':{
                        'enabled': True
                    }
            }}

**Step 2:** Specify hyperparameters, instance type and number of instances to distribute training to. 

In [5]:
instance_type_count_name = f'{instance_type.replace(".","-")}-x-{str(instance_count)}'
job_name   = f'tf-dataparallel-{instance_type_count_name}-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'

metric_definitions = [{'Name': 'Validation Accuracy', 'Regex': 'Validation Accuracy: ([0-9\\.]+)'}]

hyperparameters = {'epochs': 50, 
                   'learning-rate': 0.001,
                   'momentum': 0.95,
                   'weight-decay': 2e-4,
                   'optimizer': 'adam',
                   'batch-size' : 256,
                   'model-type': 'custom'}


In [6]:
from sagemaker.tensorflow import TensorFlow
smdp_estimator = TensorFlow(entry_point         = 'cifar10-tf2-smdataparallel.py', 
                           source_dir           = 'code',
                           output_path          = output_path + '/',
                           code_location        = output_path,
                           role                 = role,
                           instance_count       = instance_count,
                           instance_type        = instance_type,
                           framework_version    = '2.3.1', 
                           py_version           = 'py37',
                           metric_definitions   = metric_definitions,
                           hyperparameters      = hyperparameters,
                           distribution         = distribution)

**Step 4:** Specify dataset locations in Amazon S3 and then call the fit function.

In [7]:
smdp_estimator.fit({'train': datasets,
                    'validation': datasets,
                    'eval': datasets}, 
                  job_name=job_name, wait=False)